In [ ]:
%qtconsole

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
from nonlinear import hammersteinModel
from parametricaudio import ParametricAudioModel
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
output_notebook()
TOOLS = 'pan,box_zoom,wheel_zoom,reset'

In [ ]:
paramAudio = ParametricAudioModel(modulation='dsb',demodulation='square',equalize=True)
hammerMod = hammersteinModel()
n=2**12
fs=192e3
f0=1e3
t=np.arange(n)/fs

inputsignal = 2*np.random.rand(n)-1
lpf = signal.iirfilter(6,24e3/(fs/2),btype='low')
inputsignal = signal.lfilter(lpf[0], lpf[1], inputsignal)
inputsignal /= np.max(np.abs(inputsignal))



#inputsignal = signal.chirp(t,20,t[-1],24e3)
#inputsignal = np.sin(2*np.pi*f0*t)
timedata = pd.DataFrame({'input':inputsignal,'t':t})
timedata['output'] = paramAudio(inputsignal)

smoothing = np.ceil(paramAudio.smoothcycles*paramAudio.fs/paramAudio.fc).astype('int')
#inputsignal=inputsignal[smoothing:-smoothing]
#outputsignal=outputsignal[smoothing:-smoothing]
timedata.drop(timedata.head(smoothing).index,inplace=True)
timedata.drop(timedata.tail(smoothing).index,inplace=True)

hammerMod.approximateNonlinearity(timedata['input'],timedata['output'])
hammerMod.setOrthogonalBasis('legendre')
hammerMod.orthogonalSeriesExpansion(timedata['input'],timedata['output'])
hammerMod.approximateLinearity(timedata['input'],timedata['output'])

nonlinearity = pd.DataFrame({'x':np.linspace(-1,1,1e3)})
nonlinearity['rectangular'] = hammerMod.nonlinearity(nonlinearity['x'])
nonlinearity['orthogonal'] = hammerMod.orthogonalApproximation(nonlinearity['x'])


In [ ]:
f1=figure(tools=TOOLS+',box_select,lasso_select',title='Input')
f2=figure(tools=TOOLS+',box_select,lasso_select',title='Output')
f3=figure(tools=TOOLS+',box_select,lasso_select',x_axis_label='Input',y_axis_label='Output')
cds=ColumnDataSource(timedata)
f1.line(x='t',y='input',source=cds,selection_color='orange',nonselection_alpha=0.5)
f1.circle(x='t',y='input',source=cds,alpha=0,nonselection_alpha=0)
f2.line(x='t',y='output',source=cds,selection_color='orange',nonselection_alpha=0.5)
f2.circle(x='t',y='output',source=cds,alpha=0,nonselection_alpha=0)
f3.circle(x='input',y='output',source=cds,selection_color='orange')
p=gridplot([[f1],[f2],[f3]])
show(p)

In [ ]:
f=figure(tools=TOOLS,title='Nonlinear approximations')
f.line(nonlinearity['x'],nonlinearity['rectangular'],legend='Rectangular')
f.line(nonlinearity['x'],nonlinearity['orthogonal'],legend='Orthogonal',color='orange')
show(f)

In [ ]:
f=figure(tools=TOOLS,title='Impulse response')
f.line(np.arange(hammerMod.impulseResponse.size),hammerMod.impulseResponse)
show(f)